## Calculate ERPs

In [1]:
#Importing files and modules

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter, MaxNLocator
from os import path
#%matplotlib notebook
from brainpipe.system import study
from brainpipe.visual import *
from brainpipe.statistics import *
from mne.baseline import rescale
from mne.filter import filter_data
import pandas as pd
from pandas import ExcelWriter

## User variables

In [2]:
low_pass_filter = 10.
sf = 512.
norm_mode = 'mean' #'ratio' 'mean' 'percent' 
baseline = [1408,1536] #-250ms à 0ms
data_to_use = [1408, 2560]
n_perm = 200

# Compute ERPs By Odor ALL & selected trials

In [5]:
st = study('Olfacto')
path_data = path.join (st.path, 'database/TS_E_ByOdor_th40_30_250/')

subjects = ['LEFC']
#subjects = ['CHAF', 'VACJ', 'SEMC', 'PIRJ','MICP'] # 'LEFC',
nelec_odors = {
    'CHAF' : [107, ['1','2','3','4','5','7','8','9']],
    'VACJ' : [139, ['14','15','16','17','10','11','12','13']], 
    'SEMC' : [107, ['5','7','8','9','10','11','12','13']],
    'PIRJ' : [106, ['1','4','9','18','6','7',]],
    'LEFC' : [193, ['1','2','3','4','14','15','16','17']],
    'MICP' : [105, ['3','9','14','18','2','6','8','12']],
}

for su in subjects:
    print(su)
    for elec in range(nelec_odors[su][0]):
        i=-1
        j=0
        fig = plt.figure(figsize=(10,14))
        for odor in nelec_odors[su][1]:
            i=i+2
            j=j+2
            data_all = np.load(path.join(path_data, su+'_concat_odor_'+odor+'_bipo.npz'))
            data, channel, label = data_all['x'], data_all['channel'], data_all['label']
            fig.suptitle(su+' By Odor '+str(label[elec])+' ('+str(channel[elec])+') ', fontsize=22, fontweight='bold')

            # Select data for one elec + name :
            data_elec = data[elec,:,:]
            ntrials = len(data_elec[2])
            print ('Odeur : ',odor, '\nOriginal data : ', data.shape, 'Channel : ', channel[elec], 'Label : ', label[elec], 'N_trials :', ntrials, 'One elec shape : ', data_elec.shape)

            #Filter data for one elec (all trials):
            data = np.array(data_elec, dtype='float64')
            data_to_filter = np.swapaxes(data, 0, 1)
            filtered_data = filter_data(data_to_filter, sfreq=512, l_freq=None, h_freq=low_pass_filter, method='fir', phase='zero-double')
            filtered_data = np.swapaxes(filtered_data, 0, 1)
            print ('Size of filtered data:', filtered_data.shape,)

            #Normalize the non-averaged data (all trials)
            times = np.arange(filtered_data.shape[0])
            print ('time points : ', times.shape)
            filtered_data_to_norm = np.swapaxes(filtered_data, 0, 1)
            norm_filtered_data = rescale(filtered_data_to_norm, times=times, baseline=baseline, mode=norm_mode)
            norm_filtered_data = np.swapaxes(norm_filtered_data, 0, 1)
            print ('Size norm & filtered data : ', norm_filtered_data.shape,)

# =======================PREPARE DATA TO PLOT AND PLOT THE ERPs=====================================
            # Select all trials windows to plot
            all_trials_data = norm_filtered_data[range(baseline[0], data_to_use[1])]
            print('-> Shape of all trials data to plot : ', all_trials_data.shape)
            # Time vector :
            times_plot = 1000 * np.arange((baseline[0] - baseline[1]), all_trials_data.shape[0]-baseline[1] + baseline[0],) / sf
            print('-> Shape of time vector : ', times_plot.shape)
            
            #Plot all trials 
            labels_all_trials = np.arange(all_trials_data.shape[1], dtype='int64')
            fig.add_subplot(8, 2, i)
            print ('place all trials',i)
            #title = su+' Odor'+str(odor)+' '+str(ntrials)+'trials '+str(label[elec])+' ('+str(channel[elec])+') '
            color_all = ['m', 'c', 'r', 'darkorange', 'purple', 'gold', 'dimgray', 'black', 'darkcyan', 'dodgerblue',]
            BorderPlot(times_plot, all_trials_data, y=labels_all_trials, xlabel='Time (ms)',ylabel = 'Potential',
                       alpha=0.3,linewidth=2, color = color_all)
            plt.gca().yaxis.set_major_locator(MaxNLocator(3,integer=True))
            rmaxis(plt.gca(), ['right', 'top'])
            addLines(plt.gca(), vLines=[0], vColor=['firebrick']*2, vWidth=[2]*2, hLines=[0], hColor=['#000000'], hWidth=[2])

            #Plot 1st 2nd and last trials
            fig.add_subplot(8, 2, j)
            print ('3 trials place', j)
            two_trials_labels = np.arange(2, dtype='int64')
            print ('two labels size', two_trials_labels.shape)
            two_trials_data = all_trials_data[:,[0,-1]]
            #title = su+' 1st, 2nd & last trials '+str(label[elec])+' ('+str(channel[elec])+') '
            BorderPlot(times_plot, two_trials_data, y=two_trials_labels, xlabel='Time (ms)',ylabel = 'Potential',
                       alpha=0.3,linewidth=2, legend = ['1st', 'last'], color=['m','b'])
            rmaxis(plt.gca(), ['right', 'top'])
            plt.gca().yaxis.set_major_locator(MaxNLocator(3,integer=True))
            plt.xticks()
            addLines(plt.gca(), vLines=[0], vColor=['firebrick']*2, vWidth=[2]*2, hLines=[0], hColor=['#000000'], hWidth=[2])
            plt.legend(loc=0, handletextpad=0.1, frameon=False, fontsize = 'xx-small')
            #plt.show()
            del data_elec, data_all, ntrials, all_trials_data, data, data_to_filter, filtered_data, filtered_data_to_norm, norm_filtered_data, times, times_plot, labels_all_trials,
        
        #Save all the plots
        rep = path.join(st.path, 'feature/ERP_By_Odor_bipo_250ms_rescale_filtered_new/',su)
        fname = (rep + '_ERP_Odor_'+str(label[elec])+'_'+str(channel[elec])+'_('+str(elec)+').png')
        print (fname)
        #plt.show()
        plt.savefig(fname, dpi=300, bbox_inches='tight')
        plt.clf()
        plt.close
        del label, elec, channel
        

-> Olfacto loaded
LEFC


/home/karim/anaconda3/lib/python3.5/site-packages/matplotlib/pyplot.py:516: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


Odeur :  1 
Original data :  (193, 2304, 2) Channel :  a2-a1 Label :  Amg-pPirT N_trials : 2 One elec shape :  (2304, 2)
Setting up low-pass filter at 10 Hz
h_trans_bandwidth chosen to be 2.5 Hz
Filter length of 1352 samples (2.641 sec) selected
Size of filtered data: (2304, 2)
time points :  (2304,)
Applying baseline correction (mode: mean)
Size norm & filtered data :  (2304, 2)
-> Shape of all trials data to plot :  (896, 2)
-> Shape of time vector :  (896,)
place all trials 1
3 trials place 2
two labels size (2,)
Odeur :  2 
Original data :  (193, 2304, 5) Channel :  a2-a1 Label :  Amg-pPirT N_trials : 5 One elec shape :  (2304, 5)
Setting up low-pass filter at 10 Hz
h_trans_bandwidth chosen to be 2.5 Hz
Filter length of 1352 samples (2.641 sec) selected
Size of filtered data: (2304, 5)
time points :  (2304,)
Applying baseline correction (mode: mean)
Size norm & filtered data :  (2304, 5)
-> Shape of all trials data to plot :  (896, 5)
-> Shape of time vector :  (896,)
place all tri